## Download Radolan .bin.gz files from DWD ftp server

To-Do
Ordner im System erstellen mit der jeweiligen Jahreszahl der Daten
Entpacken?

In [ ]:
#!pip install pyftpdlib
import ftplib
import os

#working dir
os.chdir("/home/elisabeth/Dokumente/master/radolan_data/raw")
print(os.getcwd())

#location of the file in the ftp server
path = 'climate_environment/CDC/grids_germany/daily/radolan/historical/bin/'
host = "https://opendata.dwd.de/"
ip =  "141.38.2.26" 

#name + extension of the file you want to download form server
filename = '*.tar.gz'

#Create Connection
ftp = ftplib.FTP(ip) 

#Login
ftp.login()

#Change Path to Radolan bin directory
ftp.cwd(path)

#Get list of Years available 
paths = ftp.nlst()
print("Available Years: ", paths)

#Workaround for problem that if i cahnged first time i am not in right directory
count = True

#For loop to download all files from every year directory 
for path in paths:
    if count == True:
        count = False 
        ftp.cwd(path)
        print("Path: ", ftp.pwd())
        filenames = ftp.nlst()
        print("Files stored in this path: ", filenames)
        for file in filenames:
            handle = open(file.lstrip("/"), 'wb') #file.rstrip("/")+ "/" + 
            ftp.retrbinary('RETR %s' % file, handle.write)
            print("File " + file + " downloaded succsefully!")
        continue
    else:
        ftp.cwd(("../"))
        ftp.cwd(path)
        count = False 
        print("Path: ", ftp.pwd())
        filenames = ftp.nlst()
        print("Files stored in this path: ", filenames)
        for file in filenames:
            handle = open(file.lstrip("/"), 'wb') #file.rstrip("/")+ "/" + 
            ftp.retrbinary('RETR %s' % file, handle.write)
            print("File " + file + " downloaded succsefully!")
        continue

    
#close connection
ftp.quit()


/home/elisabeth/Dokumente/master/radolan_data/raw
Available Years:  ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
/climate_environment/CDC/grids_germany/daily/radolan/historical/bin/2006
Files stored in this path:  ['SF-200610.tar.gz', 'SF-200611.tar.gz', 'SF-200612.tar.gz']
file SF-200610.tar.gz downloaded succsefully!
file SF-200611.tar.gz downloaded succsefully!
file SF-200612.tar.gz downloaded succsefully!
2007
file : SF-200701.tar.gz
file : SF-200702.tar.gz
file : SF-200703.tar.gz
